In [ ]:
# Transcribe from microphone

In [10]:
import os
import winsound


In [1]:
import azure.cognitiveservices.speech as speechsdk


In [2]:
speech_key, service_region = "---------------------------", "-------------"


In [3]:
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)


In [4]:
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)


In [5]:
result = speech_recognizer.recognize_once()


In [6]:
if result.reason == speechsdk.ResultReason.RecognizedSpeech:
    print("Recognized: {}".format(result.text))
elif result.reason == speechsdk.ResultReason.NoMatch:
    print("No speech could be recognized: {}".format(result.no_match_details))
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech Recognition canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details: {}".format(cancellation_details.error_details))

Recognized: Hello, my name is photos.


In [ ]:
# Transcribe and translate from microphone

In [7]:
def get_result_text(reason, result):
    reason_format = {
        speechsdk.ResultReason.TranslatedSpeech:
            f'RECOGNIZED "{from_language}": {result.text}\n' +
            f'TRANSLATED into "{to_language}"": {result.translations[to_language]}',
        speechsdk.ResultReason.RecognizedSpeech: f'Recognized: "{result.text}"',
        speechsdk.ResultReason.NoMatch: f'No speech could be recognized: {result.no_match_details}',
        speechsdk.ResultReason.Canceled: f'Speech Recognition canceled: {result.cancellation_details}'
    }
    return reason_format.get(reason, 'Unable to recognize speech')

In [27]:
def translate_speech_to_text(from_language,to_language):
    translation_config = speechsdk.translation.SpeechTranslationConfig(
            subscription=speech_key, region=service_region)
    # Source (input) language
    translation_config.speech_recognition_language = from_language
    translation_config.add_target_language(to_language)
    #translation_config.add_target_language("de")
    
    # translate a given audio file
    #audio_config = speechsdk.audio.AudioConfig(filename="24 16-1215-original.wav")
    # translate from microphone
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    recognizer = speechsdk.translation.TranslationRecognizer(
            translation_config=translation_config, audio_config=audio_config)
    

    print(f'Say something in "{from_language}" and we\'ll translate into "{to_language}".')

    result = recognizer.recognize_once()
    synthesize_translations(result=result)
    print(get_result_text(reason=result.reason, result=result))

In [21]:
def synthesize_translations(result):
    language_to_voice_map = {
        "de": "de-DE-KatjaNeural",
        "en": "en-US-AriaNeural",
        "it": "it-IT-ElsaNeural",
        "pt": "pt-BR-FranciscaNeural",
        "zh-Hans": "zh-CN-XiaoxiaoNeural",
        "fr":"fr-FR-DeniseNeural"
    }
    print(f'Recognized: "{result.text}"')

    for language in result.translations:
        translation = result.translations[language]
        print(f'Translated into "{language}": {translation}')

        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
        speech_config.speech_synthesis_voice_name = language_to_voice_map.get(language)
        
        audio_config = speechsdk.audio.AudioOutputConfig(filename=f'{language}-translation.wav')
        speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
        speech_synthesizer.speak_text_async(translation).get()

In [32]:
from_language, to_language = 'en-US', 'fr'


In [33]:
translate_speech_to_text(from_language, to_language)

# Play the wav file
wavFile = to_language + "-translation.wav"
winsound.PlaySound(wavFile, winsound.SND_FILENAME)

Say something in "en-US" and we'll translate into "fr".
Recognized: "The planets revolve around the sun."
Translated into "fr": Les planètes tournent autour du soleil.
RECOGNIZED "en-US": The planets revolve around the sun.
TRANSLATED into "fr"": Les planètes tournent autour du soleil.
